In [5]:
!pip install ultralytics
!pip install roboflow
!nvidia-smi

import os
import shutil
from pathlib import Path
import yaml
from ultralytics import YOLO
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np

Tue Sep 30 18:09:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
from roboflow import Roboflow
from google.colab import userdata

api_key = userdata.get('ROBOFLOW_KEY')

rf = Roboflow(api_key=api_key)
project = rf.workspace("ufddfdd").project("ur-fall-detection-dataset")

dataset = project.version(1).download("yolov8")

# O dataset será baixado em: /content/UR-Fall-Detection-Dataset-1
dataset_path = dataset.location
print(f"Dataset baixado em: {dataset_path}")

yaml_path = f'{dataset_path}/data.yaml'
print(f"Arquivo YAML do dataset: {yaml_path}")

loading Roboflow workspace...
loading Roboflow project...
Dataset baixado em: /content/UR-Fall-Detection-Dataset-1
Arquivo YAML do dataset: /content/UR-Fall-Detection-Dataset-1/data.yaml


In [7]:
model = YOLO('yolov8n.pt')  # Modelo nano, ideal para embarcado

print("Modelo YOLOv8n carregado!")
print(f"Número de parâmetros: {sum(p.numel() for p in model.parameters()):,}")

Modelo YOLOv8n carregado!
Número de parâmetros: 3,157,200


In [8]:
training_args = {
    'data': yaml_path,           # caminho para o arquivo yaml do Roboflow
    'epochs': 50,                # reduzido para 50 épocas (dataset já é grande)
    'imgsz': 416,               # REDUZIDO para 416 para melhor FPS no Raspberry
    'batch': 16,                # batch size (ajuste conforme GPU)
    'patience': 15,             # early stopping patience
    'save': True,               # salvar checkpoints
    'device': 0,                # usar GPU (0) ou CPU ('cpu')
    'workers': 2,               # número de workers para dataloader
    'project': 'fall_detection', # nome do projeto
    'name': 'yolov8n_ur_falls', # nome do experimento
    'exist_ok': True,           # sobrescrever experimento existente
    'pretrained': True,         # usar pesos pré-treinados
    'optimizer': 'AdamW',       # otimizador
    'lr0': 0.001,              # learning rate inicial
    'lrf': 0.01,               # learning rate final (fraction of lr0)
    'momentum': 0.937,         # momentum
    'weight_decay': 0.0005,    # weight decay
    'warmup_epochs': 3.0,      # épocas de warmup
    'warmup_momentum': 0.8,    # momentum inicial no warmup
    'box': 7.5,                # peso da loss de box
    'cls': 0.5,                # peso da loss de classificação
    'dropout': 0.0,            # dropout (0.0 = desabilitado)
    'val': True,               # validar durante treinamento
    'plots': True,             # gerar plots
    'conf': 0.25,              # threshold de confiança para validação
    'iou': 0.45,               # threshold IoU para NMS na validação
    'augment': True,           # ativar augmentação de dados
}

print("Configuração otimizada para Raspberry Pi:")
print("- Imagem: 416x416 (melhor balanço FPS/precisão)")
print("- Modelo: YOLOv8n (nano)")
print("- Dataset: UR Fall Detection (3,065 imagens)")


Configuração otimizada para Raspberry Pi:
- Imagem: 416x416 (melhor balanço FPS/precisão)
- Modelo: YOLOv8n (nano)
- Dataset: UR Fall Detection (3,065 imagens)


In [9]:
results = model.train(**training_args)

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/UR-Fall-Detection-Dataset-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.45, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_ur_falls, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=15, perspecti

In [10]:
results_path = 'fall_detection/yolov8n_falls'

# Confusion Matrix
if os.path.exists(f'{results_path}/confusion_matrix.png'):
    display(Image(f'{results_path}/confusion_matrix.png'))

# Curvas de treinamento
if os.path.exists(f'{results_path}/results.png'):
    display(Image(f'{results_path}/results.png'))

# F1 curve
if os.path.exists(f'{results_path}/F1_curve.png'):
    display(Image(f'{results_path}/F1_curve.png'))

# PR curve
if os.path.exists(f'{results_path}/PR_curve.png'):
    display(Image(f'{results_path}/PR_curve.png'))

In [12]:
best_model = YOLO('fall_detection/yolov8n_ur_falls/weights/best.pt')

# Validar no conjunto de teste
metrics = best_model.val(
    data=yaml_path,
    split='test',
    imgsz=640,
    batch=16,
    conf=0.25,
    iou=0.7,
    device=0
)

print("Métricas de validação:")
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.mp:.3f}")
print(f"Recall: {metrics.box.mr:.3f}")

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1007.0±400.6 MB/s, size: 40.5 KB)
val: Scanning /content/UR-Fall-Detection-Dataset-1/test/labels... 200 images, 4 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 200/200 2.2Kit/s 0.1s
val: New cache created: /content/UR-Fall-Detection-Dataset-1/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 13/13 2.7it/s 4.9s
                   all        200        196      0.957      0.983      0.988      0.838
                  fall         76         76      0.947      0.974      0.981      0.863
                person        120        120      0.968      0.992      0.994      0.812
Speed: 2.7ms preprocess, 8.2ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to /content/runs/detect/val
Métricas de 

In [17]:
!git clone https://github.com/ekramalam/GMDCSA24-A-Dataset-for-Human-Fall-Detection-in-Videos.git /content/inference


Cloning into '/content/inference'...
remote: Enumerating objects: 302, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 302 (delta 16), reused 38 (delta 6), pack-reused 237 (from 1)
Receiving objects: 100% (302/302), 1.05 GiB | 40.56 MiB/s, done.
Resolving deltas: 100% (40/40), done.
Updating files: 100% (170/170), done.


In [18]:
import os
from pathlib import Path
from ultralytics import YOLO
from google.colab import files

print("✅ Bibliotecas importadas e setup inicial concluído.")

DATASET_ROOT = Path("/content/inference")
MODEL_PATH = Path("/content/fall_detection/yolov8n_ur_falls/weights/best.pt")
OUTPUT_ROOT = Path("/content/results_inference")

OUTPUT_ROOT.mkdir(exist_ok=True)

if not DATASET_ROOT.exists():
    print(f"❌ ERRO: A pasta do dataset '{DATASET_ROOT}' não foi encontrada!")
elif not MODEL_PATH.exists():
    print(f"❌ ERRO: O modelo '{MODEL_PATH}' não foi encontrado! Faça o upload do seu 'best.pt'.")
else:
    print(f"▶️ Pasta do dataset encontrada em: {DATASET_ROOT}")
    print(f"▶️ Modelo a ser usado: {MODEL_PATH}")
    print(f"▶️ Resultados serão salvos em: {OUTPUT_ROOT}")

    try:
        model = YOLO(MODEL_PATH)
        print("\n✅ Modelo YOLO carregado com sucesso!")
    except Exception as e:
        print(f"❌ ERRO ao carregar o modelo: {e}")
        model = None

    if model:
        video_files = list(DATASET_ROOT.rglob('*.mp4'))
        print(f"✅ Encontrados {len(video_files)} vídeos para processar.")

        # --- 4. PROCESSAR CADA VÍDEO ---
        if not video_files:
            print("⚠️ Nenhum vídeo .mp4 foi encontrado no diretório do dataset.")
        else:
            print("\n🚀 Iniciando o processo de inferência em todos os vídeos...")
            for i, video_path in enumerate(video_files):
                print(f"\n--- Processando vídeo {i+1}/{len(video_files)}: {video_path.name} ---")

                relative_path = video_path.relative_to(DATASET_ROOT)
                output_path_run = OUTPUT_ROOT / relative_path.parent

                try:
                    results = model.predict(
                        source=video_path,
                        save=True,
                        stream=True,
                        project=output_path_run,
                        name=video_path.stem
                    )


                    for r in results:
                        pass

                    print(f"✅ Vídeo '{video_path.name}' processado. Resultado salvo.")

                except Exception as e:
                    print(f"❌ ERRO ao processar o vídeo {video_path.name}: {e}")

            print("\n🎉 Processo de inferência concluído para todos os vídeos!")
            print(f"👉 Verifique os resultados na pasta: {OUTPUT_ROOT}")

A saída de streaming foi truncada nas últimas 5000 linhas.
video 1/1 (frame 11/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 13.5ms
video 1/1 (frame 12/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 10.7ms
video 1/1 (frame 13/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 9.6ms
video 1/1 (frame 14/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 15.7ms
video 1/1 (frame 15/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 16.0ms
video 1/1 (frame 16/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 1 person, 10.1ms
video 1/1 (frame 17/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 2 persons, 9.7ms
video 1/1 (frame 18/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 1 person, 11.5ms
video 1/1 (frame 19/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 1 person, 9.5ms
video 1/1 (frame 20/382) /content/inference/Subject 4/ADL/10.mp4: 256x416 1 person, 13.2ms
video 1/1 (frame 21/382) /co

In [2]:
from ultralytics import YOLO
best_model = YOLO('fall_detection/yolov8n_ur_falls/weights/best.pt')

# Modificando o modelo para TENSORLITE para o raspberry
yaml_path = '/content/UR-Fall-Detection-Dataset-1/data.yaml'

best_model.export(
    format='tflite',
    imgsz=640,
    int8=True,
    data=yaml_path
)

print("\nModelo exportado com sucesso!")
print("Arquivos salvos em: fall_detection/yolov8n_falls/weights/")

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'fall_detection/yolov8n_ur_falls/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.9 MB)
requirements: Ultralytics requirement ['onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 6.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.19.1rc1 opset 22...
ONNX: slimming with onnxslim 0.1.70...
ONNX: export success ✅ 1.9s, saved as 'fall_detection/yolov8n_ur_falls/weights/best.onnx' (11.8 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=/content/UR-Fall-Detection-Dataset-1/data.yaml'
Fast image access ✅ (ping: 0.0±0.0 ms, read: 19.4±2.1 MB/s, size: 39.4 KB)
Scanning /